In [1]:
from src.bgc_providers.ohio_bgc_provider import OhioBgcProvider
from src.mongo import MongoDB
import json
from time import sleep
from loguru import logger

In [2]:
mongo = MongoDB()
mongo.ping()

2023-08-20 08:06:10.849 | DEBUG    | src.mongo:ping:18 - Pinged your deployment. You successfully connected to MongoDB!


In [3]:
test_provider = OhioBgcProvider()
test_dataframe = test_provider.tsfresh_dataframe()
print(test_dataframe.head(5))
stream = test_provider.simulate_glucose_stream()
next(stream)
next(stream)
next(stream)
next(stream)

2023-08-20 08:06:13.694 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:17:00', 'value': '101'}
2023-08-20 08:06:13.694 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:22:00', 'value': '98'}
2023-08-20 08:06:13.694 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:27:00', 'value': '104'}
2023-08-20 08:06:13.704 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:32:00', 'value': '112'}


            date_time   mock_date time_of_day part_of_day  time  bg_value id
0 2021-12-07 01:17:00  2021-12-07    01:17:00  late night  0.00       101  a
1 2021-12-07 01:22:00  2021-12-07    01:22:00  late night  0.08        98  a
2 2021-12-07 01:27:00  2021-12-07    01:27:00  late night  0.17       104  a
3 2021-12-07 01:32:00  2021-12-07    01:32:00  late night  0.25       112  a
4 2021-12-07 01:37:00  2021-12-07    01:37:00  late night  0.33       120  a


{'timestamp': 1638840720.0,
 'time': '2021-12-07T01:32:00+00:00',
 'value': 112.0,
 'patient': '559'}

In [4]:
mongo.list_databases()
mongo.list_collections('test_database_1')

2023-08-20 08:06:30.168 | INFO     | src.mongo:list_databases:24 - Found 3 databases: ['test_database_1', 'admin', 'local']
2023-08-20 08:06:30.234 | INFO     | src.mongo:list_collections:30 - Found 6 collections in test_database_1: ['posts', 'cgm_1', 'system.buckets.cgm_1', 'cgm_2', 'system.buckets.cgm_2', 'sandalphon1']


['posts',
 'cgm_1',
 'system.buckets.cgm_1',
 'cgm_2',
 'system.buckets.cgm_2',
 'sandalphon1']

In [7]:
db = mongo.client['test_database_1']
cgm_db = db['cgm_3']

In [9]:
try:
    while True:
        values = next(stream)
        logger.info(values)
        rec_id = cgm_db.insert_one(values).inserted_id
        logger.info(rec_id)
        sleep(5)
except KeyboardInterrupt:
    print('Interrupted by the user')

2023-08-20 08:09:33.335 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:47:00', 'value': '135'}
2023-08-20 08:09:33.338 | INFO     | __main__:<module>:4 - {'timestamp': 1638841620.0, 'time': '2021-12-07T01:47:00+00:00', 'value': 135.0, 'patient': '559'}
2023-08-20 08:09:33.509 | INFO     | __main__:<module>:6 - 64e1a00dca5affdf6d035168
2023-08-20 08:09:38.512 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:52:00', 'value': '142'}
2023-08-20 08:09:38.512 | INFO     | __main__:<module>:4 - {'timestamp': 1638841920.0, 'time': '2021-12-07T01:52:00+00:00', 'value': 142.0, 'patient': '559'}
2023-08-20 08:09:38.624 | INFO     | __main__:<module>:6 - 64e1a012ca5affdf6d035169
2023-08-20 08:09:43.634 | INFO     | src.bgc_providers.ohio_bgc_provider:simulate_glucose_stream:49 - {'ts': '07-12-2021 01:57:00', 'value': '140'}
2023-08-20 08:09:43.634 | INFO     | __main__:<module>:4 - {'timestamp':

Interrupted by the user
